In [12]:
import pandas as pd
from scipy.io import arff

# Đọc file ARFF
data, meta = arff.loadarff('airlines.arff')

# Chuyển đổi thành DataFrame
df = pd.DataFrame(data)

# Hiển thị DataFrame
df.head()

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,b'CO',269.0,b'SFO',b'IAH',b'3',15.0,205.0,b'1'
1,b'US',1558.0,b'PHX',b'CLT',b'3',15.0,222.0,b'1'
2,b'AA',2400.0,b'LAX',b'DFW',b'3',20.0,165.0,b'1'
3,b'AA',2466.0,b'SFO',b'DFW',b'3',20.0,195.0,b'1'
4,b'AS',108.0,b'ANC',b'SEA',b'3',30.0,202.0,b'0'


In [13]:
# Chuyển đổi các cột từ kiểu byte sang kiểu string
df['Airline'] = df['Airline'].str.decode('utf-8')
df['AirportFrom'] = df['AirportFrom'].str.decode('utf-8')
df['AirportTo'] = df['AirportTo'].str.decode('utf-8')
df['DayOfWeek'] = df['DayOfWeek'].str.decode('utf-8')
df['Delay'] = df['Delay'].str.decode('utf-8')

# Chuyển đổi cột 'Delay' sang kiểu int
df['Delay'] = df['Delay'].astype(int)

# Hiển thị thông tin sau khi tiền xử lý
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Airline      539383 non-null  object 
 1   Flight       539383 non-null  float64
 2   AirportFrom  539383 non-null  object 
 3   AirportTo    539383 non-null  object 
 4   DayOfWeek    539383 non-null  object 
 5   Time         539383 non-null  float64
 6   Length       539383 non-null  float64
 7   Delay        539383 non-null  int32  
dtypes: float64(3), int32(1), object(4)
memory usage: 30.9+ MB


,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,CO,269.0,SFO,IAH,3,15.0,205.0,1
1,US,1558.0,PHX,CLT,3,15.0,222.0,1
2,AA,2400.0,LAX,DFW,3,20.0,165.0,1
3,AA,2466.0,SFO,DFW,3,20.0,195.0,1
4,AS,108.0,ANC,SEA,3,30.0,202.0,0


In [14]:
from sklearn.preprocessing import LabelEncoder

# Mã hóa các biến phân loại
label_encoders = {}
categorical_columns = ['Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek']

for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le


# Hiển thị DataFrame sau khi tiền xử lý
df

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,4,269.0,253,135,2,15.0,205.0,1
1,14,1558.0,217,60,2,15.0,222.0,1
2,1,2400.0,154,80,2,20.0,165.0,1
3,1,2466.0,253,80,2,20.0,195.0,1
4,2,108.0,14,252,2,30.0,202.0,0
...,...,...,...,...,...,...,...,...
539378,4,178.0,203,264,4,1439.0,326.0,0
539379,8,398.0,252,16,4,1439.0,305.0,0
539380,8,609.0,253,184,4,1439.0,255.0,0
539381,13,78.0,128,253,4,1439.0,313.0,1


In [15]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [16]:
def create_datastream(data, batch_size):
    datastream = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size]
        datastream.append(batch)
    return datastream

In [17]:
def train_random_forest_classifier(X, y, n_estimators=50):
    """
    Huấn luyện bộ phân loại Random forest trên dữ liệu X, y.
    n_estimators: Số lượng cây trong Random forest.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf

In [18]:
def adaptive_ensemble_size(C, sample, unique_label, anpha, min_num=3):
    """
    Cơ chế adaptive_esemble 
    C : Bộ phân loại yếu
    sample
    unique_label : Tập hợp nhãn trong luồng dữ liệu
    anpha : hyper parameter
    min_num : số lượng C tối thiểu 
    """
    probability_list = []
    # Tính toán xác suất dự đoán cho mỗi bộ phân loại trong C và lưu trữ kết quả
    sample_proba = [clf.predict_proba(sample)[0] for clf in C]
    
    for label in unique_label:
        li = []
        latest_proba = None
        
        for proba in sample_proba:
            # Tạo từ điển để ánh xạ nhãn với xác suất tương ứng
            probability_dict = {lbl: prob for lbl, prob in zip(C[0].classes_, proba)}
            # Lấy xác suất hiện tại cho nhãn đang xét
            current_proba = probability_dict.get(label, 0)
            
            if len(li) < min_num:
                # Nếu số lượng xác suất trong li nhỏ hơn min_num, thêm current_proba vào li
                li.append(current_proba)
            else:
                if latest_proba is None:
                    # Nếu latest_proba chưa được gán giá trị, gán giá trị của nó bằng xác suất mới nhất
                    latest_proba = probability_dict.get(label, 0)
                
                # So sánh xác suất hiện tại với xác suất mới nhất
                if abs(current_proba - latest_proba) < anpha:
                    # Nếu chênh lệch nhỏ hơn anpha, thêm current_proba vào li
                    li.append(current_proba)
                else:
                    # Nếu chênh lệch lớn hơn anpha, dừng vòng lặp
                    break
        
        # Thêm danh sách xác suất cho nhãn hiện tại vào probability_list
        probability_list.append(li)
    
    return probability_list

In [19]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    
    for i, li in enumerate(probability_list):
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        slope, intercept = linear_regression(x, y)
        next_value = slope * (len(li) + 2) + intercept
        li = [next_value] + li  # Thêm giá trị dự đoán tiếp theo vào đầu danh sách
        weights = 1 - np.arange(1, len(li) + 1) * epsilon
        weighted_prob = np.dot(li, weights) / len(li)
        predicted_probabilities.append(weighted_prob)
    
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps


In [20]:
def process_data_stream(S, m, k, unique_labels):
    C = deque(maxlen=m)
    true_labels = []
    predicted_labels = []
    pre = []
    block_accuracies = []  # Danh sách để lưu trữ độ chính xác của từng block

    for i in range(len(S) - 1):
        Bi = S[i]
        block_predictions = []
        X = Bi.iloc[:, :-1]
        y = Bi.iloc[:, -1]
        Ci = train_random_forest_classifier(X, y)
        C.append(Ci)
        if len(C) < k:
            continue
        Bi_1 = S[i + 1]
        for index, row in Bi_1.iterrows():
            sample = pd.DataFrame([row[:-1]], columns=Bi.columns[:-1])
            anpha = (1500 / len(Bi_1)) * 0.2
            selected_classifiers = adaptive_ensemble_size(C, sample, unique_labels, anpha)
            pre_sample = tendency_prediction(selected_classifiers, unique_labels)
            block_predictions.append(pre_sample)
            true_labels.append(row.iloc[-1])
            predicted_labels.append(pre_sample)
        pre.append(block_predictions)

        # Tính toán độ chính xác cho block hiện tại
        block_accuracy = accuracy_score(true_labels[-len(Bi_1):], predicted_labels[-len(Bi_1):])
        block_accuracies.append(block_accuracy)
    
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Accuracy: {accuracy}")

    # Trực quan hóa độ chính xác của từng block
    plt.figure(figsize=(10, 6))
    plt.plot(block_accuracies, marker='o', linestyle='-', color='b')
    plt.title('Accuracy of Each Block')
    plt.xlabel('Block Index')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.show()

# Main execution
batch_size = 1500
S = create_datastream(df, batch_size)
m = 15
k = 3
unique_labels = list(set(df.iloc[:, -1]))
process_data_stream(S, m, k, unique_labels)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

def train_random_forest_classifier(X, y):
    clf = RandomForestClassifier()
    clf.fit(X, y)
    return clf

def process_data_stream(S):
    """
    Processes data stream S using a Random Forest classifier.
    - S: Data stream (list of data blocks Bi)
    """
    # Use the first block as the training set
    Bi = S[0]
    X_train = Bi.iloc[:, :-1]
    y_train = Bi.iloc[:, -1]
    model = train_random_forest_classifier(X_train, y_train)

    accuracies = []

    # Predict and calculate accuracy for each subsequent block
    for i in range(1, len(S)):
        Bi_1 = S[i]
        X_test = Bi_1.iloc[:, :-1]
        y_test = Bi_1.iloc[:, -1]
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    average_accuracy = sum(accuracies) / len(accuracies)
    print(average_accuracy)
    # Plot the accuracies
    plt.plot(range(1, len(S)), accuracies, marker='o')
    plt.xlabel('Block Index')
    plt.ylabel('Accuracy')
    plt.title('Accuracy for Each Block')
    plt.show()

process_data_stream(S) 